In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np

from generate.vae_ import VariationalAutoEncoder
from generate.recons_utils import get_reconstructed_df
from generate.corr_loss_ import correlation_loss
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

In [2]:
file_name = "10ft_12k"
df = pd.read_csv(f"data/input_reals/{file_name}.csv", ";")
df

C:\Users\murat\AppData\Local\Temp\ipykernel_10416\947115369.py:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(f"data/input_reals/{file_name}.csv", ";")


,CROSSING,FINISSHING,HEADING_ACCURACY,SHORT_PASSING,VOLLEYS,DRIBBLING,CURVE,FREE_KICK_ACCURACY,LONG_PASSING,BALL_CONTROL
0,56.0,28.0,54.0,68.0,55.0,58.0,58.0,49.0,65.0,62.0
1,21.0,20.0,26.0,27.0,21.0,30.0,8.0,11.0,75.0,34.0
2,49.0,67.0,47.0,73.0,59.0,71.0,63.0,62.0,67.0,72.0
3,41.0,62.0,60.0,60.0,52.0,70.0,49.0,38.0,48.0,68.0
4,60.0,41.0,46.0,70.0,60.0,62.0,47.0,31.0,68.0,67.0
...,...,...,...,...,...,...,...,...,...,...
11995,82.0,70.0,55.0,81.0,69.0,86.0,85.0,77.0,79.0,85.0
11996,61.0,52.0,66.0,74.0,67.0,66.0,45.0,67.0,71.0,72.0
11997,66.0,85.0,81.0,57.0,76.0,77.0,69.0,50.0,32.0,76.0
11998,69.0,55.0,60.0,75.0,54.0,66.0,53.0,59.0,72.0,71.0


In [3]:
ORIGINAL_DIM = df.shape[1]
LATENT_DIM = 16
EPOCHS = 100
BATCH_SIZE = 64
COLS = df.columns

vae = VariationalAutoEncoder(
    original_dim = ORIGINAL_DIM,
    latent_dim = LATENT_DIM)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()
cos_loss_fn = tf.keras.losses.CosineSimilarity()
# corr_loss_fn = correlation_loss
loss_metric = tf.keras.metrics.Mean()

In [4]:
# todo -1 +1 scaling ile sigmoid (0, 1 ) fail

In [5]:
scaler = MinMaxScaler((-1, 1))
scaled_original_df = scaler.fit_transform(df)
df_tensor = tf.convert_to_tensor(scaled_original_df, dtype=tf.float32)

train_dataset = tf.data.Dataset.from_tensor_slices(df_tensor)
train_dataset_batch = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

In [ ]:
# Iterate over epochs.
for epoch in range(EPOCHS):
    print(f"\n--> epoch : {epoch}")
    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset_batch):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            # Compute reconstruction loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            mse_los = mse_loss_fn(x_batch_train, reconstructed)
            loss += -1*(correlation_loss(x_batch_train, reconstructed))
            corr_loss = -1*(correlation_loss(x_batch_train, reconstructed))
            loss += cos_loss_fn(x_batch_train, reconstructed)
            cos_loss = cos_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses) # Add KLD regularization loss
        
        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))
        loss_metric(loss)
        
        if step % 30 == 0:
            print(f"step {step}: mean loss = {loss_metric.result()}")
            corlos = np.array(corr_loss).mean()
            coslos = np.array(cos_loss).mean()
            mselos = np.array(mse_los).mean()
            print(f"CORR : {corlos} - COS : {coslos} - MSELOSS : {mselos}")

#     print(f"MSE ", mse_loss)
#     print(f"CORR ", corr_loss)
#     print(f"COS ", cos_loss)
#     print(f"KLD ", kld_loss)


--> epoch : 0
step 0: mean loss = 0.14094208180904388
CORR : 0.10106421262025833 - COS : -0.14605437219142914 - MSELOSS : 0.18527553975582123
step 30: mean loss = -0.6414874792098999
CORR : -0.44841986894607544 - COS : -0.5168237686157227 - MSELOSS : 0.1921352744102478
step 60: mean loss = -0.7776511311531067
CORR : -0.5560397505760193 - COS : -0.7303314805030823 - MSELOSS : 0.10308738797903061
step 90: mean loss = -0.9066787362098694
CORR : -0.7032454013824463 - COS : -0.7813883423805237 - MSELOSS : 0.06626001745462418
step 120: mean loss = -0.9954851269721985
CORR : -0.7074287533760071 - COS : -0.8305098414421082 - MSELOSS : 0.05947594344615936
step 150: mean loss = -1.0529381036758423
CORR : -0.7592859268188477 - COS : -0.8377558588981628 - MSELOSS : 0.05441942438483238
step 180: mean loss = -1.096389651298523
CORR : -0.7836712598800659 - COS : -0.8289076089859009 - MSELOSS : 0.04575446993112564

--> epoch : 1
step 0: mean loss = -1.1052078008651733
CORR : -0.7464379072189331 - COS

In [6]:
reconstructed_df = get_reconstructed_df(vae, scaler, train_dataset_batch, COLS)

reconstructed_df

,CROSSING,FINISSHING,HEADING_ACCURACY,SHORT_PASSING,VOLLEYS,DRIBBLING,CURVE,FREE_KICK_ACCURACY,LONG_PASSING,BALL_CONTROL
0,73.978569,67.407524,43.894798,78.179070,65.925720,80.118034,71.326698,74.741295,71.209610,79.607597
1,53.290607,42.589695,85.193840,63.924442,54.491467,42.075859,38.887478,49.708366,58.591904,60.550339
2,65.049141,73.812294,73.006737,70.938904,66.663841,71.741310,65.006538,46.295261,59.324646,73.237885
3,71.250496,49.118912,67.516998,67.396942,47.310799,60.963017,53.595688,51.075912,63.324348,64.740074
4,58.389687,78.869888,75.294052,68.686920,78.589455,76.032204,68.016685,56.121864,48.170811,76.061760
...,...,...,...,...,...,...,...,...,...,...
2995,62.779560,51.403713,67.343758,70.214630,44.028263,61.535179,52.878155,57.362133,64.678703,66.338615
2996,47.529564,31.337271,74.956299,59.293949,31.688738,44.386929,35.243267,30.522829,54.596981,57.118431
2997,51.741638,38.122543,71.718681,70.226303,39.329273,54.713417,36.257950,45.639908,65.294281,65.285858
2998,11.942747,15.308197,13.581575,21.430750,13.456026,13.960362,10.551446,17.410881,45.424591,16.536169


In [8]:
reconstructed_df.round().to_csv(f"data/output_synt/{file_name}.csv", sep=";", index=False)